In [2]:
!pip install wandb
import wandb
from wandb.keras import WandbMetricsLogger
wandb.init()

import tensorflow as tf

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

--2024-01-11 14:15:07--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2024-01-11 14:15:07 (19.4 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [33]:
# Get TensorFlow Datasets
import tensorflow_datasets as tfds


# Load in the data (takes about 5-6 minutes in Google Colab)
train_data, test_data = tfds.load(name="food101", split=["train", "validation"], shuffle_files=True, as_supervised=True)

In [34]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=(224,224)):
    image      = tf.image.resize(image, img_shape)
    img_tensor = tf.cast(image, tf.float32)

    return img_tensor, label


In [35]:
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data  = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data  = test_data.batch(batch_size=32).prefetch(tf.data.AUTOTUNE)

In [36]:
train_data, test_data

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>)

In [37]:
# Create TensorBoard callback (already have "create_tensorboard_callback()" from a previous notebook)
from helper_functions import create_tensorboard_callback

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath = "model_checkpoints/cp.ckpt", monitor="val_accuracy",
                                                      save_best_only=True, save_weights_only=True, verbose=2)

In [38]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16") # set global policy to mixed precision

In [39]:
class Custom_Model_Class(tf.keras.Model):
    def __init__(self):
        super().__init__()

        base_model           = tf.keras.applications.EfficientNetB0(include_top=False)
        base_model.trainable = False # freeze base model layers

        INPUT_SHAPE          = (224, 224, 3)
        input_layer  = tf.keras.layers.Input(shape=INPUT_SHAPE, name="input_layer")
        embedding    = tf.keras.layers.GlobalAveragePooling2D()
        fc1          = tf.keras.layers.Dense(101)
        activation   = tf.keras.layers.Activation("softmax", dtype=tf.float32)

        self.complete_model = tf.keras.models.Sequential([
            input_layer,

            base_model,
            embedding,
            fc1,
            activation
        ])

    def call(self, input_batch):
        final_output_probs = self.complete_model(input_batch)

        return final_output_probs

model = Custom_Model_Class()
model.compile(loss="sparse_categorical_crossentropy", # Use sparse_categorical_crossentropy when labels are *not* one-hot
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [26]:
# 1. Create a function to recreate the original model
def create_model():
  # Create base model
  input_shape = (224, 224, 3)
  base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False)
  base_model.trainable = False # freeze base model layers

  # Create Functional model
  inputs = layers.Input(shape=input_shape, name="input_layer")
  # Note: EfficientNetBX models have rescaling built-in but if your model didn't you could have a layer like below
  # x = layers.Rescaling(1./255)(x)
  x = base_model(inputs, training=False) # set base_model to inference mode only
  x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
  x = layers.Dense(101)(x) # want one output neuron per class
  # Separate activation of output layer so we can output float32 activations
  outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
  model = tf.keras.Model(inputs, outputs)

  return model

# 2. Create and compile a new version of the original model (new weights)
model = create_model()
model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

In [40]:
# Fit the model with callbacks
history_101_food_classes_feature_extract = model.fit(train_data, epochs=3, validation_data=test_data,
                                                     steps_per_epoch=len(train_data), validation_steps=int(0.15 * len(test_data)),
                                                     callbacks=[create_tensorboard_callback("training_logs", "efficientnetb0_101_classes_all_data_feature_extract"),
                                                                model_checkpoint])

model.save("07_efficientnetb0_feature_extract_model_mixed_precision")
# loaded_saved_model = tf.keras.models.load_model(save_dir)

Saving TensorBoard log files to: training_logs/efficientnetb0_101_classes_all_data_feature_extract/20240111-150034
Epoch 1/3
2368/2368 [==============================] - ETA: 0s - loss: 1.7739 - accuracy: 0.5692
Epoch 1: val_accuracy improved from -inf to 0.69359, saving model to model_checkpoints/cp.ckpt
2368/2368 [==============================] - 174s 69ms/step - loss: 1.7739 - accuracy: 0.5692 - val_loss: 1.1343 - val_accuracy: 0.6936
Epoch 2/3
2368/2368 [==============================] - ETA: 0s - loss: 1.2648 - accuracy: 0.6723
Epoch 2: val_accuracy improved from 0.69359 to 0.71875, saving model to model_checkpoints/cp.ckpt
2368/2368 [==============================] - 177s 74ms/step - loss: 1.2648 - accuracy: 0.6723 - val_loss: 1.0336 - val_accuracy: 0.7188
Epoch 3/3
2368/2368 [==============================] - ETA: 0s - loss: 1.1292 - accuracy: 0.7051
Epoch 3: val_accuracy improved from 0.71875 to 0.73040, saving model to model_checkpoints/cp.ckpt
2368/2368 [====================

In [ ]:
# Evaluate model (unsaved version) on whole test dataset
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

## Fine Tuning, existing trained model

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=3) # if val loss decreases for 3 epochs in a row, stop training

# Create ModelCheckpoint callback to save best model during fine-tuning
checkpoint_path = "fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

In [ ]:
# Download the saved model from Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip
# Unzip the SavedModel downloaded from Google Stroage
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model

# Load and evaluate downloaded GS model
loaded_gs_model = tf.keras.models.load_model("downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:


# Creating learning rate reduction callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=2,
                                                 verbose=1, # print out when learning rate goes down
                                                 min_lr=1e-7)

# Compile the model
loaded_gs_model.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                        metrics=["accuracy"])

# Start to fine-tune (all layers)
history_101_food_classes_all_data_fine_tune = loaded_gs_model.fit(train_data,
                                                        epochs=100, # fine-tune for a maximum of 100 epochs
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)), # validation during training on 15% of test data
                                                        callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"), # track the model training logs
                                                                   model_checkpoint, # save only the best model during training
                                                                   early_stopping, # stop model after X epochs of no improvements
                                                                   reduce_lr]) # reduce the learning rate after X epochs of no improvements